In [38]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.utils import to_categorical, plot_model
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator

import cv2

#MLP ON CIFAR

# network parameters
batch_size = 128
num_classes = 10
epochs = 25
data_augmentation = True


# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
image_size = x_train.shape[1]
# print('xtrain shape: ', x_train[0].shape)
# import matplotlib.pyplot as plt
# plt.imshow(x_train[0])
# plt.show()
# cv2.imshow("train", x_train[0])
# cv2.waitKey(0)
# cv2.destroyAllWindows()
x_train = x_train.reshape(-1, 3072)
x_test = x_test.reshape(-1, 3072)
x_train = x_train.astype('float32') / 255;
x_test = x_test.astype('float32') / 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
input_size = 3072
max_batches = len(x_train) / batch_size


# Convert class vectors to binary class matrices.
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# model is a 3-layer MLP with ReLU and dropout after each layer
model = Sequential()
model.add(Dense(512, input_dim=input_size))
model.add(Activation('relu'))
#model.add(Dropout(0.40))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.08))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.12))
model.add(Dense(num_classes))
# this is the output for one-hot vector
model.add(Activation('softmax'))
model.summary()
plot_model(model, to_file='mlp-cifar.png', show_shapes=True)

# loss function for one-hot vector
# use of sgd optimizer with default lr=0.01
# accuracy is good metric for classification tasks
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    # train the network no data augmentation
    x_train = np.reshape(x_train, [-1, input_size])
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    # we need [width, height, channel] dim for data aug
    x_train = np.reshape(x_train, [-1, image_size, image_size, 3])
    datagen = ImageDataGenerator(
        data_format="channels_last",
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0.0,  # randomly rotate images in the range (deg 0 to 180)
        width_shift_range=0.0,  # randomly shift images horizontally
        height_shift_range=0.0,  # randomly shift images vertically
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    datagen.fit(x_train)
    for e in range(epochs):
        print("Epoch: %d/%d" % (e+1, epochs))
        batches = 0
        for x_batch, y_batch in datagen.flow(x_train, y_train, batch_size=batch_size):
            x_batch = np.reshape(x_batch, [-1, input_size])
            if e == 24:
                model.fit(x_batch, y_batch, verbose=1)
            else:
                model.fit(x_batch, y_batch, verbose=0)
            batches += 1
            if batches >= max_batches:
                # we need to break the loop by hand because
                # the generator loops indefinitely
                break

# Score trained model.
x_test = np.reshape(x_test, [-1, input_size])
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


x_train shape: (50000, 3072)
50000 train samples
10000 test samples
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_137 (Dense)            (None, 512)               1573376   
_________________________________________________________________
activation_137 (Activation)  (None, 512)               0         
_________________________________________________________________
dense_138 (Dense)            (None, 512)               262656    
_________________________________________________________________
activation_138 (Activation)  (None, 512)               0         
_________________________________________________________________
dropout_77 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_139 (Dense)            (None, 512)               262656    
_________________________________________________________________
activati

**MLP on CIFAR10**


---


*Test 1*

128 | relu

128 | relu

128 | relu

Optimizer: SGD

Epoch20 accuracy: 0.5106

Test accuracy: 48.7%

---

*Test 2*

128 | relu

128 | relu

128 | relu

Optimizer: Adam

Epoch20 accuracy: 0.5704

Test accuracy: 51.0%


---

*Test 3*

386 | relu | dropout

386 | relu

386 | relu | dropout

Optimizer: Adam

Epoch20 accuracy: 0.5549

Test accuracy: 50.5%


---

**network parameters**

batch_size = 128
hidden_units = 390
dropout = 0.16
num_classes = 10
epochs = 25

Epoch 25/25
50000/50000 [==============================] - 4s 87us/step - loss: 1.3945 - acc: 0.4975

10000/10000 [==============================] - 2s 201us/step

Test accuracy: 49.1%



---
**network parameters**

batch_size = 64
hidden_units = 256
dropout = 0.12
num_classes = 10
epochs = 25

Epoch 25/25
50000/50000 [==============================] - 7s 133us/step - loss: 1.4289 - acc: 0.4887
10000/10000 [==============================] - 3s 253us/step

Test accuracy: 48.4%

---

**network parameters**
batch_size = 256
hidden_units = 386
dropout = 0.12
num_classes = 10
epochs = 25

Epoch 25/25
50000/50000 [==============================] - 2s 49us/step - loss: 1.2511 - acc: 0.5504
10000/10000 [==============================] - 2s 231us/step

Test accuracy: 52.0%

---






In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import cv2

from keras.layers import Dense, Dropout, Input
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.models import Model
from keras.layers.merge import concatenate
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.utils import plot_model

#CNN ON CIFAR

# load MNIST dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# from sparse label to categorical
num_labels = len(np.unique(y_train))
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# reshape and normalize input images
image_size = x_train.shape[1]
x_train = np.reshape(x_train,[-1, image_size, image_size, 3])
x_test = np.reshape(x_test,[-1, image_size, image_size, 3])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

#network parameters
input_shape = (image_size, image_size, 3)
batch_size = 32
kernel_size = 3
dropout = 0.38
n_filters = 32

# left branch of Y network
left_inputs = Input(shape=input_shape)
x = left_inputs
filters = n_filters
# 3 layers of Conv2D-Dropout-MaxPooling2D
# number of filters doubles after each layer (32-64-128)
for i in range(3):
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               padding='same',
               activation='relu')(x)
    x = Dropout(dropout)(x)
    x = MaxPooling2D()(x)
    filters *= 2

# right branch of Y network
right_inputs = Input(shape=input_shape)
y = right_inputs
filters = n_filters
# 3 layers of Conv2D-Dropout-MaxPooling2D
# number of filters doubles after each layer (32-64-128)
for i in range(3):
    y = Conv2D(filters=filters,
               kernel_size=kernel_size,
               padding='same',
               activation='relu',
               dilation_rate=2)(y)
    y = Dropout(dropout)(y)
    y = MaxPooling2D()(y)
    filters *= 2

# merge left and right branches outputs
y = concatenate([x, y])
# feature maps to vector in preparation to connecting to Dense layer
y = Flatten()(y)
y = Dropout(dropout)(y)
outputs = Dense(num_labels, activation='softmax')(y)

# build the model in functional API
model = Model([left_inputs, right_inputs], outputs)
# verify the model using graph
plot_model(model, to_file='cnn-y-network.png', show_shapes=True)
# verify the model using layer text description
model.summary()

# classifier loss, Adam optimizer, classifier accuracy
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# train the model with input images and labels
model.fit([x_train, x_train],
          y_train, 
          validation_data=([x_test, x_test], y_test),
          epochs=20,
          batch_size=batch_size)

# model accuracy on test dataset
score = model.evaluate([x_test, x_test], y_test, batch_size=batch_size)
print("\nTest accuracy: %.1f%%" % (100.0 * score[1]))

Using TensorFlow backend.


170500096/170498071 [==============================] - 29s 0us/step
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 32)   896         input_1[0][0]                    
____________________________________________________

**CNN on CIFAR10**

*Test 1 (Siamese Network)*

128 | relu

128 | relu

128 | relu

Optimizer: adam

Epoch20 accuracy: 0.6630

Test accuracy: 66.3%

---

